### Graded Lab 5

Hello ! Welcome to Graded Lab of Module 5.

In the last assignment we had worked on missing values & outliers.
Its high time when we look at the feature engineering & Principal component analysis.

**So lets go ahead with the last graded assignment for the course !**

In case if you are not able to recollect the problem description and data description then mentioning it below.

Lets look at the problem statement,

*Client: ABC Retail, Incorporated, rest-of-the-world division* 

***Project name: Online retail sales analysis*** 

An online retailer, ABC, Inc., operates in nearly 100 countries worldwide, selling furniture, office supplies and technology products to customers in three segments: consumer, corporate and home office. ABC, Inc. is a US-based company, and it has two major divisions: US and rest of the world. We are working with the rest of the world division of the company. 

They have provided us with online sales transaction data from 2011 to 2014.

We are given 3 datasets:-

1. Data on each sale; 51290 records; all data in US dollars
It contains fields like
**order_id** (identifier) ,order_date ,ship_date ,ship_mode ,**customer_id**(identifier) ,product_id ,category ,sub_category ,product_name ,sales ,quantity ,discount ,profit ,shipping_cost ,order_priority ,**vendor_code** (identifier) 


2. Data on the customers; 1590 records 
It contains fields like
**customer_id** (identifier) ,customer_name ,city ,state ,country ,postal_code ,segment ,market ,region 

3. Data on vendors who supply the retailer; 65 records 
It contains fields like
vendor ,**vendor_code** (identifier) 

We need to analyze the data and need to provide answer to different questions asked by company officials.

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from itertools import combinations
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### Reading sales data
sales = pd.read_csv('sales_data.csv')

### Reading customer data
cust = pd.read_csv(r'customers.csv',encoding='iso-8859-1')

### Reading vendor data
vend = pd.read_csv(r'vendors.csv')
sales.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,vendor_code
0,1,MX-2014-143658,02/10/2014,06/10/2014,Standard Class,SC-20575,OFF-LA-10002782,Office Supplies,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.0,4.56,1.03,Medium,VE_001
1,2,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,FUR-FU-10004015,Furniture,Furnishings,"Tenex Clock, Durable",252.16,8,0.0,90.72,13.45,Medium,VE_002
2,3,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,FUR-BO-10002352,Furniture,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.0,54.08,9.63,Medium,VE_003
3,4,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,OFF-BI-10004428,Office Supplies,Binders,"Cardinal Binder, Clear",35.44,4,0.0,4.96,1.37,Medium,VE_004
4,5,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,OFF-AR-10004594,Office Supplies,Art,"Sanford Canvas, Water Color",71.60,2,0.0,11.44,3.79,Medium,VE_005


In order to solve the next questions , we need to combine all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. So here we have written a data processing function.
There are 2 tasks which are to be performed.
1. Merge/ Join all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. (Understand which should be the joining key , type of join , refer .merge() function of pandas)
2. Convert 'order_date' into a datetime column.
**Return output as a dataframe**

In [2]:
#### data_merging & order_date processing , data1 will be sales , data2 will be customer dataset & data3 will be vendor dataset.

def data_process(data1,data2,data3):
    # your code here
    merged_df = pd.merge(data1,data2,on='customer_id',how='left')
    merged_df = pd.merge(merged_df,data3,on='vendor_code',how='left')
    merged_df['order_date'] = pd.to_datetime(merged_df['order_date'])
    return merged_df

In [3]:
sales= data_process(data1=sales.copy(),data2=cust.copy(),data3=vend.copy())

In [4]:
assert sales['order_date'].dtypes=='<M8[ns]' ,'Make sure if you have converted order_date into a datetime format correctly or not.'
assert sales.shape== (51290,26) ,'Checking size and shape of dataframe after merging is a very important check.'

### Q.1) Create a feature for identifying which orders are ordered on weekends. Flag will take value 1 if order is ordered on weekends & 0 otherwise. (Sat & Sun are weekends). Return  the output series.

Hint: Think of using weekday() function, consider'order_date' as the required columns for computations. 

In [5]:
sales['weekend_orders'] = sales['order_date'].dt.day_name().map(lambda x: 1 if (x=='Saturday' or x =='Sunday') else 0)
sales.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,...,customer_name,city,state,country,postal_code,segment,market,region,vendor,weekend_orders
0,1,MX-2014-143658,2014-02-10,06/10/2014,Standard Class,SC-20575,OFF-LA-10002782,Office Supplies,Labels,"Hon File Folder Labels, Adjustable",...,Sonia Cooley,Mexico City,Distrito Federal,Mexico,NaN,Consumer,LATAM,LATAM-North,Accenco,0
1,2,MX-2012-155047,2012-10-15,20/10/2012,Standard Class,KW-16570,FUR-FU-10004015,Furniture,Furnishings,"Tenex Clock, Durable",...,Kelly Williams,Dos Quebradas,Risaralda,Colombia,NaN,Consumer,LATAM,LATAM-South,Ace Corporation,0
2,3,MX-2012-155047,2012-10-15,20/10/2012,Standard Class,KW-16570,FUR-BO-10002352,Furniture,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",...,Kelly Williams,Dos Quebradas,Risaralda,Colombia,NaN,Consumer,LATAM,LATAM-South,Nero,0
3,4,MX-2012-155047,2012-10-15,20/10/2012,Standard Class,KW-16570,OFF-BI-10004428,Office Supplies,Binders,"Cardinal Binder, Clear",...,Kelly Williams,Dos Quebradas,Risaralda,Colombia,NaN,Consumer,LATAM,LATAM-South,Rushcorp,0
4,5,MX-2012-155047,2012-10-15,20/10/2012,Standard Class,KW-16570,OFF-AR-10004594,Office Supplies,Art,"Sanford Canvas, Water Color",...,Kelly Williams,Dos Quebradas,Risaralda,Colombia,NaN,Consumer,LATAM,LATAM-South,Low Tide Corp,0


In [6]:
def weekend_flag(data):
    # your code here
    data['weekend_orders'] = data['order_date'].dt.day_name().map(lambda x: 1 if (x=='Saturday' or x =='Sunday') else 0)
    return data['weekend_orders']
    
weekend_flag(data=sales.copy()).unique()

array([0, 1])

In [7]:
sales['weekend_orders']=weekend_flag(data=sales)

In [8]:
# autograder cell , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert all(sales['weekend_orders'].unique()==np.array([0, 1])) ,'Make sure that you have created flag correctly'


### Q.2) We wish to create a feature called 'discount_per_quantity'. Write a functional code to compute ratio of 2 columns. Return  the output series.

In [9]:
sales.columns

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'product_id', 'category', 'sub_category', 'product_name',
       'sales', 'quantity', 'discount', 'profit', 'shipping_cost',
       'order_priority', 'vendor_code', 'customer_name', 'city', 'state',
       'country', 'postal_code', 'segment', 'market', 'region', 'vendor',
       'weekend_orders'],
      dtype='object')

In [10]:
sales.apply(lambda row: row['discount']/row['quantity'],axis=1).sample(100).head(5)

32064    0.100000
19277    0.000000
38374    0.022222
2727     0.000000
6144     0.001000
dtype: float64

In [11]:
def feature_1(data,col1,col2):
    # your code here
    col3 = data.apply(lambda row: row[col1]/row[col2],axis=1)
    return col3

In [12]:
sales['discount_per_quantity']= feature_1(data=sales,col1='discount',col2='quantity')

In [13]:
# autograder cell , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert round(sales['discount_per_quantity'].mean(),4)==0.0643,'Make sure that you have created feature correctly.'


### Q.3) Create a feature for finding cumulative sales for every customer. Return  the output series.

In [14]:
sales.columns

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'product_id', 'category', 'sub_category', 'product_name',
       'sales', 'quantity', 'discount', 'profit', 'shipping_cost',
       'order_priority', 'vendor_code', 'customer_name', 'city', 'state',
       'country', 'postal_code', 'segment', 'market', 'region', 'vendor',
       'weekend_orders', 'discount_per_quantity'],
      dtype='object')

In [15]:
data = sales.sort_values(['customer_id','order_date'],ascending=[1,1],inplace=True)
data = sales[['customer_id','order_date','sales']]
data.head(10)

,customer_id,order_date,sales
33525,AA-10315,2011-03-31,673.5680
33526,AA-10315,2011-03-31,52.9800
27383,AA-10315,2011-04-07,51.7110
27384,AA-10315,2011-04-07,44.9295
27385,AA-10315,2011-04-07,29.2380
27386,AA-10315,2011-04-07,278.8500
38764,AA-10315,2011-09-15,14.9400
38765,AA-10315,2011-09-15,14.5600
7997,AA-10315,2011-11-24,339.3600
7998,AA-10315,2011-11-24,9.0600


In [16]:
temp = data.groupby(['customer_id'])['sales'].cumsum()
temp

33525     673.5680
33526     726.5480
27383     778.2590
27384     823.1885
27385     852.4265
           ...    
39637    8100.9629
20429    8546.2889
15730    8762.2889
15731    9350.3114
15732    9479.3444
Name: sales, Length: 51290, dtype: float64

In [17]:
def cumulative_sales(data):
    # your code here
    data.sort_values(['customer_id','order_date'],ascending=[1,1],inplace=True)
    return data.groupby(['customer_id'])['sales'].cumsum()

In [18]:
### Sorting the dataframe wrto customer_id & order_date
sales.sort_values(['customer_id','order_date'],ascending=[1,1],inplace=True)
### Applying cumulative_sales function
sales['cumulative_sales']=cumulative_sales(data=sales)

In [19]:
# autograder cell , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert round(sales[sales['customer_id']=='ZC-21910']['cumulative_sales'].max(),2)==28472.82

### Q.4) Please demonstrate how to enhance a given dataset by introducing a new feature that records the frequency of same-day purchases made by each customer. The desired outcome should be a modified dataframe containing a variable named 'same_day_purchase_frequency' which accurately portrays the same-day purchase behavior of individual customers.


Hint:

1.Same_day_purchase_frequency will be the frequency of the orders made on same day. If customer 'ABC' has placed 2 orders on 2nd Jan 23 then 'same_day_purchase_frequency'for this customer for 2nd Jan 23 will be 2. This is still at Customer * Date level

2.If customer  'ABC' has made say 10 such frequency transactions where frequency of same day purchase >1. Then'same_day_frequency_purchase' for customer will be 10. This is our final answer at Customer level.  

In [20]:
sales.head(2)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,...,state,country,postal_code,segment,market,region,vendor,weekend_orders,discount_per_quantity,cumulative_sales
33525,33526,CA-2011-128055,2011-03-31,05/04/2011,Standard Class,AA-10315,OFF-BI-10004390,Office Supplies,Binders,GBC DocuBind 200 Manual Binding Machine,...,Sinaloa,Mexico,NaN,Consumer,LATAM,LATAM-North,Lifoods,0,0.1,673.568
33526,33527,CA-2011-128055,2011-03-31,05/04/2011,Standard Class,AA-10315,OFF-AP-10002765,Office Supplies,Appliances,Fellowes Advanced Computer Series Surge Protec...,...,Sinaloa,Mexico,NaN,Consumer,LATAM,LATAM-North,Lifoods,0,0.0,726.548


In [21]:
data = sales.copy()
# Group by 'customer_id' and date, and count the orders, store count of orders in variable named 'order_count'
order_counts = data.groupby(['customer_id','order_date']).agg({'order_id':'count'})
order_counts = order_counts.reset_index().rename(columns={'order_id':'order_count'})
                                                               
# Filter for same-day purchases (order_count > 1)
same_day_purchases = order_counts[order_counts['order_count'] > 1]

# Group by 'customer_id' and calculate the same-day purchase frequency , name that column as 'same_day_purchase_frequency'
same_day_purchase_frequency = same_day_purchases.groupby('customer_id')['order_count'].count()
same_day_purchase_frequency = same_day_purchase_frequency.reset_index().rename(columns={'order_count':'same_day_purchase_frequency'})

# Merge the result with the original DataFrame to add the same-day purchase frequency as a new column  
data = pd.merge(data,same_day_purchase_frequency,on='customer_id',how='left').reset_index(drop=True)
data.head()
# Fill missing values with 0 (for customers with no same-day purchases)
data['same_day_purchase_frequency'].fillna(0,inplace=True)

In [22]:
data.isnull().sum()

row_id                             0
order_id                           0
order_date                         0
ship_date                          0
ship_mode                       1570
customer_id                        0
product_id                         0
category                           0
sub_category                       0
product_name                    1297
sales                              0
quantity                           0
discount                           0
profit                             0
shipping_cost                      0
order_priority                  1046
vendor_code                        0
customer_name                      0
city                               0
state                              0
country                            0
postal_code                    51290
segment                            0
market                             0
region                             0
vendor                             0
weekend_orders                     0
d

In [23]:
def same_day_purchase(data):
    # your code here
    
    # Group by 'customer_id' and date, and count the orders, store count of orders in variable named 'order_count'
    order_counts = data.groupby(['customer_id','order_date']).agg({'order_id':'count'})
    order_counts = order_counts.reset_index().rename(columns={'order_id':'order_count'})

    # Filter for same-day purchases (order_count > 1)
    same_day_purchases = order_counts[order_counts['order_count'] > 1]

    # Group by 'customer_id' and calculate the same-day purchase frequency , name that column as 'same_day_purchase_frequency'
    same_day_purchase_frequency = same_day_purchases.groupby('customer_id')['order_count'].count()
    same_day_purchase_frequency = same_day_purchase_frequency.reset_index().rename(columns={'order_count':'same_day_purchase_frequency'})

    # Merge the result with the original DataFrame to add the same-day purchase frequency as a new column  
    data = pd.merge(data,same_day_purchase_frequency,on='customer_id',how='left').reset_index(drop=True)
    data.head()
    # Fill missing values with 0 (for customers with no same-day purchases)
    data['same_day_purchase_frequency'].fillna(0,inplace=True)

    return data

In [24]:
sales_new=same_day_purchase(data=sales.copy())
# autograder cell , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert float(sales_new[sales_new['customer_id']=='ZC-11910']['same_day_purchase_frequency'].unique())==0,'Make sure that you have created variable correctly.'


### Q.5) Create a variable which will calculate weighted sales per market. Return  the output series.
Weights for every market are :-'LATAM':1, 'EMEA':1.3, 'Africa':2.7, 'Canada':1.4, 'EU':2.8, 'APAC':3.5

In [25]:
sales.market.unique()

array(['LATAM', 'EMEA', 'Africa', 'Canada', 'EU', 'APAC'], dtype=object)

In [26]:
data = sales.copy()
import pandas as pd
import numpy as np

data = data.assign(weighted_market_sales=lambda x: np.where(x['market'] == 'EMEA', x['sales']*1.3,
                                                       np.where(x['market'] == 'Africa', x['sales']*2.7,
                                                               np.where(x['market'] == 'Canada', x['sales']*1.4,
                                                                       np.where(x['market'] == 'EU', x['sales']*2.8,
                                                                               np.where(x['market'] == 'APAC', x['sales']*3.5,x['sales']))))))


data[['sales','market','weighted_market_sales']].sample(500).head()

,sales,market,weighted_market_sales
38284,1158.120,LATAM,1158.1200
5839,42.120,LATAM,42.1200
44690,91.989,Africa,248.3703
19997,311.472,LATAM,311.4720
45257,75.180,Africa,202.9860


In [27]:
def weighted_sales(data):
    # Write a lambda function & apply it to dataframe.Kindly ignore raiseNotImplementError
    # your code here
    import pandas as pd
    import numpy as np
    data = data.assign(
            weighted_market_sales=
                lambda x: np.where(x['market'] == 'EMEA', x['sales']*1.3,
                               np.where(x['market'] == 'Africa', x['sales']*2.7,
                                       np.where(x['market'] == 'Canada', x['sales']*1.4,
                                               np.where(x['market'] == 'EU', x['sales']*2.8,
                                                       np.where(x['market'] == 'APAC', x['sales']*3.5,x['sales']))))))

    return data['weighted_market_sales']

In [28]:
sales.columns

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'product_id', 'category', 'sub_category', 'product_name',
       'sales', 'quantity', 'discount', 'profit', 'shipping_cost',
       'order_priority', 'vendor_code', 'customer_name', 'city', 'state',
       'country', 'postal_code', 'segment', 'market', 'region', 'vendor',
       'weekend_orders', 'discount_per_quantity', 'cumulative_sales'],
      dtype='object')

In [29]:
sales['weighted_market_sales']=weighted_sales(data=sales)

In [30]:
# autograder cell , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert round(float(sales[(sales['market']=='Africa') & (round(sales['sales'],4)==58.83)]['weighted_market_sales'].unique()),2)==158.84, 'Make sure that ypu have created column correctly.'

Most of the variables in our data are categorical in nature & for further anlysis purpose if we convert them by ordinal encoding or one hot encoding, we will be left with too many variables.
If we have too many variables in the data , it might cause trouble in later stages like machine learning model building.
1. Having too many variables can lead to overfitting and poor generalization to new data.
2. It can make it difficult to analyze and interpret the data and identify the most important variables.
3. It can increase computational complexity and time required to build and train a model.

How to deal with this ?

Wait , we have learned about **Principal component analysis** , we can leverage that method here for **dimensionality reduction.**

### Q.6 ) Complete the following code for PCA implementation. Return the no of PC's required for capturing the 80% of variation, variance explained by the PC's .

In [60]:
data = sales.copy()
target=0.80

# import numpy as np
# from sklearn.preprocessing import OrdinalEncoder
# priority = ['Lowest','Low','Medium','High','Critical','nan_val']
# enc = OrdinalEncoder(categories=priority)
# data['order_priority'].fillna('nan_val',inplace=True)


# Treating ordinal variable ,
# order_priority : 'Medium':2, 'Low':1, 'High':3, 'Critical':4 
data['order_priority_encoded']=data['order_priority'].map(
    lambda x: 2 if x=='Medium' else 1 if x=='Low' else 3 if x=='High' else 4 if x=='Critical' else x) # Write your code in place of None

# Treating missing values
# Impute the missing values in 'order_prioirty_encoded','ship_mode' by mode, 
data['order_priority_encoded']=data['order_priority_encoded'].fillna(data['order_priority_encoded'].mode()[0]) # Write your code in place of None
data['ship_mode']=data['ship_mode'].fillna(data['ship_mode'].mode()[0]) # Write your code in place of None


# Treating Nominal variables ,'ship_mode','category','sub_category' (while encoding them make sure to select k-1 dummy variables for a variable with k categories)
data=pd.get_dummies(data,columns=['ship_mode','category','sub_category'], drop_first=True) # Write your code in place of None

# Final data selection
# Select all the variables with 'sales','quantity','discount','profit','shipping_cost','order_priority_encoded' & dummies in 'ship_mode','category','sub_category'.
data_1=data[['sales','quantity','discount','profit','shipping_cost','order_priority_encoded','ship_mode_Same Day','ship_mode_Second Class','ship_mode_Standard Class','category_Office Supplies','category_Technology','sub_category_Appliances','sub_category_Art','sub_category_Binders','sub_category_Bookcases','sub_category_Chairs','sub_category_Copiers','sub_category_Envelopes','sub_category_Fasteners','sub_category_Furnishings','sub_category_Labels','sub_category_Machines','sub_category_Paper','sub_category_Phones','sub_category_Storage','sub_category_Supplies','sub_category_Tables']]


# Standardize the data
sc = StandardScaler()

cols = ['sales','quantity','discount','profit','shipping_cost','order_priority_encoded']
for col in cols:
    data_1[col] = sc.fit_transform(data_1[col].values.reshape(-1,1)) # Write your code in place of None


from sklearn.decomposition import PCA

# Create a PCA instance    
pca = PCA()

# Fit PCA to the scaled data
pca.fit(data_1) # Write your code in place of None

#  Calculate cumulative explained variance
explained_variance_ratio = pca.explained_variance_ratio_ # Write your code in place of None
cumulative_variance = np.cumsum(explained_variance_ratio) # Write your code in place of None


target_variation=target
n_components = np.argmax(cumulative_variance >= target_variation) + 1


# Excellent we got no of pcs which are required for capturing the target variation in the data

### Fitting PCA again with no of pc's we got above.
pca = PCA(n_components=n_components)

# Fit PCA to the scaled data
pca.fit(data_1) # Write your code in place of None

#  Calculate cumulative explained variance
explained_variance_ratio = pca.explained_variance_ratio_ # Write your code in place of None

In [62]:
def PCA(data,target=0.80):
    # your code here
    
    # Treating ordinal variable ,
    # order_priority : 'Medium':2, 'Low':1, 'High':3, 'Critical':4 
    data['order_priority_encoded']=data['order_priority'].map(
        lambda x: 2 if x=='Medium' else 1 if x=='Low' else 3 if x=='High' else 4 if x=='Critical' else x) # Write your code in place of None

    # Treating missing values
    # Impute the missing values in 'order_prioirty_encoded','ship_mode' by mode, 
    data['order_priority_encoded']=data['order_priority_encoded'].fillna(data['order_priority_encoded'].mode()[0]) # Write your code in place of None
    data['ship_mode']=data['ship_mode'].fillna(data['ship_mode'].mode()[0]) # Write your code in place of None


    # Treating Nominal variables ,'ship_mode','category','sub_category' (while encoding them make sure to select k-1 dummy variables for a variable with k categories)
    data=pd.get_dummies(data,columns=['ship_mode','category','sub_category'], drop_first=True) # Write your code in place of None

    # Final data selection
    # Select all the variables with 'sales','quantity','discount','profit','shipping_cost','order_priority_encoded' & dummies in 'ship_mode','category','sub_category'.
    data_1=data[['sales','quantity','discount','profit','shipping_cost','order_priority_encoded','ship_mode_Same Day','ship_mode_Second Class','ship_mode_Standard Class','category_Office Supplies','category_Technology','sub_category_Appliances','sub_category_Art','sub_category_Binders','sub_category_Bookcases','sub_category_Chairs','sub_category_Copiers','sub_category_Envelopes','sub_category_Fasteners','sub_category_Furnishings','sub_category_Labels','sub_category_Machines','sub_category_Paper','sub_category_Phones','sub_category_Storage','sub_category_Supplies','sub_category_Tables']]


    # Standardize the data
    sc = StandardScaler()

    cols = ['sales','quantity','discount','profit','shipping_cost','order_priority_encoded']
    for col in cols:
        data_1[col] = sc.fit_transform(data_1[col].values.reshape(-1,1)) # Write your code in place of None


    from sklearn.decomposition import PCA

    # Create a PCA instance    
    pca = PCA()

    # Fit PCA to the scaled data
    pca.fit(data_1) # Write your code in place of None

    #  Calculate cumulative explained variance
    explained_variance_ratio = pca.explained_variance_ratio_ # Write your code in place of None
    cumulative_variance = np.cumsum(explained_variance_ratio) # Write your code in place of None


    target_variation=target
    n_components = np.argmax(cumulative_variance >= target_variation) + 1


    # Excellent we got no of pcs which are required for capturing the target variation in the data

    ### Fitting PCA again with no of pc's we got above.
    pca = PCA(n_components=n_components)

    # Fit PCA to the scaled data
    pca.fit(data_1) # Write your code in place of None

    #  Calculate cumulative explained variance
    explained_variance_ratio = pca.explained_variance_ratio_ # Write your code in place of None
       
    return n_components,explained_variance_ratio

In [63]:
[round(x,4) for x in list(PCA(data=sales.copy())[1])]

[0.3012, 0.1487, 0.1395, 0.1094, 0.0724, 0.0413]

In [64]:
# autograder cell , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert type(PCA(data=sales.copy())[0])==np.int64 , 'Number of components required for capturing target variation should be in integer format.'
assert type(PCA(data=sales.copy())[1])==np.ndarray , 'Type of variation captured should be in an array format.'
assert [round(x,4) for x in list(PCA(data=sales.copy())[1])][0]==0.3012, 'Make sure that you are returning correct values for output.'


### Q.7) How much variation was explained by 3rd principal component ? 
**Output should be rounded upto 4 digits**

In [65]:
def var_expl(variation_explained):
    # Kindly ignore raiseNotImplementError   
    # your code here
    explained_variance_ratio = [round(x,4) for x in list(PCA(data=sales.copy())[1])]
    return explained_variance_ratio[2]
    

In [ ]:
# autograder cell , please do not alter/ delete /edit this cell,Kindly ignore this cell.